In [24]:
import cv2
import numpy as np

video_file = r"D:\My_Projects\GOCV\video\Video_20230920115011534.avi"  # Update with your video file path
cap = cv2.VideoCapture(video_file)
#out=cv2.VideoWriter('output.mp4', cap, 20.0, (1500, 800))

if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7,7), 0)
    _, thresh = cv2.threshold(blurred, 115, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #frame_with_contours = cv2.drawContours(frame, contours, -1, (255, 0, 0), 2)

    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=100, param2=30, minRadius=72, maxRadius=80
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle
            cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)

            # Countours detection enclosed by circle
            count = 0
            for contour in contours:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                distance = np.sqrt((i[0] - center[0])**2 + (i[1] - center[1])**2)
                if distance <= i[2]:
                    count += 1
                    cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)

                # Display the count and condition result
                if count <= 30:
                   condition_result = "Good"
                else:
                   condition_result = "Not Good"
                        
            # Display the count
            cv2.putText(frame, f"Count: {count}", (i[0] - 20, i[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            #cv2.putText(frame, f"Condition: {condition_result}", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 4, cv2.LINE_AA) #(i[0] - 20, i[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    #out.write(frame)
    cv2.imshow("Logitech Brio Webcam", frame)

    if cv2.waitKey(150) & 0xFF == ord('q'):
        break

#out.release()
cap.release()
cv2.destroyAllWindows()
